# Random Forest


<img src="./images/forest.jpg" style="width:100%"/>

Random forest is a Supervised Machine Learning Algorithm that is used widely in Classification and Regression problems. 

One of the most important features of the Random Forest Algorithm is that it can handle data sets containing continuous and categorical variables for both regression and classification problems.

### What is a Decision Tree? 

From a high level we can define Decision Trees as algorithms that can be used both classification and regression tasks providing a model to make predictions based on a universe of labeled training data.

A decision tree consists of several layers of IF-THEN-ELSE "forks" that are generated automatically during the training phase to fit the provided data.

#### An example of a decision tree

This decision tree classifies a numeric value as a probability, a percentage or invalid.

<img src="./images/decision-tree-example.png" style="width:500px"/>


## RandomForest

Since the decision tree can become very sensible to bad or inconsistent data and 
outliers the random forest algorithm creates and trains multiple decision trees 
which are combined to a single response taking their winner vote for classification or the average value  for regression problems.

<img src="./images/random-forest-example.png" style="width:600px"/>


#### Used only for tabular data

Random forests can only work with structured (tabular) data, meaning data that can be represented
as csv or any other similar technique. Unstructured data like images, speech or text cannot be
processed from random forest and most likely they will require some form of a Neural Network like CNN
or RNN for example.

#### Data requirements

Although "by book" Random Forests should be able to handle both missing and nominal
data the library that is widely used in the python world (SkLearn) does not comply
with it and in order to use it we will need to:

- Clean missing values (either removing them or adjusting them)
- Convert nominal data to categorical
- Unlike to NN in random forests there in no need to normalize or scale the data

#### Optimizing Scikit Random Forest Models

While the internal model parameters are learned during the 
training phase the hyperparameters must be specified in advance
(and this is why fine tuning a model is not exact science but it
involves a great deal of trial and error attempts).

Specifically for Random Forests two of the most important
hyperparameters to use are the following:

- The max number of decirsion trees in the forest
- The max number of features to use in an IF-THEN-ELSE case

The full description of what can be parameterized using
the Scikit implemenation see the
[officical documenenation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).


**Easy to understand and interpret**

In contrary to NN or most of the other ML algorithms the random forest 
results to an easier to visualize model.  Of course in real world problems
even random forests might become hairy and difficult to follow but it in 
simpler problems they can be viewed in a intuitive friendly way.

**Inference friendly**

Based on it architecture having a trained decision tree model makes it very
easy to measure the significance of each feature.

**Not very sensitive to outliers**

Unlike decision trees the random forests are relatively stable in turns of outliers
since they consist of a large number of trees that tend to mitigate their impact.


**Memory intensive**

During the training phase it is possible for large amounts of memory to be 
required especially as the number of features grows.

**Training can become slow**

As the sample population grows both in terms of rows and features the algorithm
can become slow since it needs to process a large number of underlined trees.

**Biased Feature Importances**

Although we can easily see the importance of each feature it is also possible 
to reach a biased conclusion in the case that we have very few distinct and
correlated values.


### Example

The iris data set that we have already solve using a neural network is ideal
for a simple example of random forest as well. The main reason for this is that
all the features are numeric and the output consists of class (in other words
it is a classifier) so we can just use the dataset as is without the need to
preprosessing it.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Load the data and randomize them.
df = pd.read_csv("./data/iris-dataset.csv")
df = df.sample(frac=1)
display(df)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
59,60,5.2,2.7,3.9,1.4,versicolor
98,99,5.1,2.5,3.0,1.1,versicolor
86,87,6.7,3.1,4.7,1.5,versicolor
11,12,4.8,3.4,1.6,0.2,setosa
52,53,6.9,3.1,4.9,1.5,versicolor
...,...,...,...,...,...,...
81,82,5.5,2.4,3.7,1.0,versicolor
87,88,6.3,2.3,4.4,1.3,versicolor
133,134,6.3,2.8,5.1,1.5,virginica
100,101,6.3,3.3,6.0,2.5,virginica


## Convert targets to categorical

Since the target is a nominal value we need to convert it to categorical 
to make it compatible with the Sklearn implementation of Random Forest 
that we will use.

In [3]:
column_name = "Species"
temp = df.copy()
dummies = pd.get_dummies(temp[column_name], prefix=column_name)
new_frame = pd.concat( [temp, dummies], axis=1, join="inner")
df = new_frame.drop(columns=[column_name, "Id"])
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_setosa,Species_versicolor,Species_virginica
59,5.2,2.7,3.9,1.4,0,1,0
98,5.1,2.5,3.0,1.1,0,1,0
86,6.7,3.1,4.7,1.5,0,1,0
11,4.8,3.4,1.6,0.2,1,0,0
52,6.9,3.1,4.9,1.5,0,1,0
...,...,...,...,...,...,...,...
81,5.5,2.4,3.7,1.0,0,1,0
87,6.3,2.3,4.4,1.3,0,1,0
133,6.3,2.8,5.1,1.5,0,0,1
100,6.3,3.3,6.0,2.5,0,0,1


The following picture can help us understand the objective of our model now that we have expanded the nominal targets to numberics:

<img src="./images/iris-input-output.png" style="width:60%"/>


## Separate the inputs and the outputs

The inputs (also known as features) must be contained in their own
data frame and the same applies to the outputs (labels) as well.

In [4]:
output_cols = ["Species_setosa", "Species_versicolor", "Species_virginica"]
X = df.copy()
Y = X[output_cols].copy()
X = X.drop(output_cols, axis=1)

display(X)
display(Y)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
59,5.2,2.7,3.9,1.4
98,5.1,2.5,3.0,1.1
86,6.7,3.1,4.7,1.5
11,4.8,3.4,1.6,0.2
52,6.9,3.1,4.9,1.5
...,...,...,...,...
81,5.5,2.4,3.7,1.0
87,6.3,2.3,4.4,1.3
133,6.3,2.8,5.1,1.5
100,6.3,3.3,6.0,2.5


,Species_setosa,Species_versicolor,Species_virginica
59,0,1,0
98,0,1,0
86,0,1,0
11,1,0,0
52,0,1,0
...,...,...,...
81,0,1,0
87,0,1,0
133,0,0,1
100,0,0,1


## Split the data to two train and test

The training data (consisting both of X and Y values) are used to train the algoritm 
while the testing data are neven seen from the training algorithm and we use them
to see how well our model performs.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print("Number of rows")
print("Train", X_train.shape[0])
print("Test", X_test.shape[0])

Number of rows
Train 120
Test 30


## Create and train the model

In [6]:
clf = RandomForestClassifier(n_estimators=100, max_features='sqrt')
clf.fit(X_train, y_train.values)

RandomForestClassifier(max_features='sqrt')

## Find the accuracy of the model

In [7]:
accuracy_rf = clf.score(X_test, y_test)
print(f'accuracy: {accuracy_rf}')

accuracy: 1.0


# A visual representation of our model

<img src="./images/iris-decission-tree.png" style="width:60%"/>